In [1]:
!pip install transformers torch
!pip install underthesea


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 72.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.8 MB/s eta 0:00:00


In [3]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report
from tqdm import tqdm
from underthesea import word_tokenize

In [5]:
# # Tải PhoBERT tokenizer và model từ Hugging Face
# tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
# phobert_model = AutoModel.from_pretrained("vinai/phobert-base")


In [6]:
data = pd.read_csv('/kaggle/input/tk2-deepleaning/df_final (1).csv')
# Hàm xử lý văn bản sử dụng Underthesea để tách từ
def preprocess_text(text):
    # Tách từ với underthesea
    text = word_tokenize(text, format="text")
    return text
data['cleaned_old_data'] = data['cleaned_old_data'].astype(str)
data = data[data['cleaned_old_data'].notnull()]
# Áp dụng tiền xử lý lên tập dữ liệu trước khi mã hóa bằng PhoBERT
data['cleaned_old_data'] = data['cleaned_old_data'].apply(preprocess_text)

data

,Unnamed: 0,cleaned_old_data,label
0,0,ngày lần hai nhất ít_định mua bạn nếu ok đều t...,2
1,1,đáng rất giao khi trước kiểm_khách cho gửi tin...,1
2,2,rồi muốn quá thấy về mới không hàng bảo tgdd t...,0
3,3,tiền đồng gần siêu còn chụp tới camera đúng xà...,1
4,4,em bán luôn người khó_thoại điện_xài mà lớn cho,0
...,...,...,...
71269,71269,pin mua bảo_tem là vậy_phẩm sao_hành nhưng phò...,5
71270,71270,lâu ít rồi còn bóc mình dùng nữa thích đã nên ...,5
71271,71271,khách khi hơn tốt nên chuyển nhưng để tâm chốn...,5
71272,71272,yếu thì mua_định như khách_hàng mới về hơn với...,5


In [7]:
X_train, X_test, y_train, y_test = train_test_split(data['cleaned_old_data'], data['label'], test_size=0.2, random_state=42)


In [8]:
# Tạo từ điển từ vựng từ tập dữ liệu
vocab = {}
index = 1
for sentence in data['cleaned_old_data']:
    for word in sentence.split():
        if word not in vocab:
            vocab[word] = index
            index += 1
vocab['<PAD>'] = 0  # Thêm token đặc biệt cho padding

# Hàm chuyển câu thành chỉ số từ vựng
def sentence_to_indices(sentence, vocab, max_len=256):
    tokens = sentence.split()
    indices = [vocab.get(token, vocab['<PAD>']) for token in tokens]
    # Padding để tất cả các câu có độ dài giống nhau
    return indices + [vocab['<PAD>']] * (max_len - len(indices))

# Chuyển dữ liệu thành các chỉ số
X_train_indices = torch.tensor([sentence_to_indices(sentence, vocab) for sentence in X_train])
X_test_indices = torch.tensor([sentence_to_indices(sentence, vocab) for sentence in X_test])


In [10]:
# # Hàm mã hóa văn bản
# def encode_texts(texts, tokenizer, max_length=256):
#     inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=max_length)
#     return inputs

# # Mã hóa dữ liệu
# X_train_encoded = encode_texts(X_train.tolist(), tokenizer)
# X_test_encoded = encode_texts(X_test.tolist(), tokenizer)

# Chuyển đổi nhãn từ pandas Series sang tensor PyTorch
y_train = torch.tensor(y_train.values)
y_test = torch.tensor(y_test.values)


In [11]:
# # Tạo DataLoader cho tập train và test
# train_data = TensorDataset(X_train_encoded['input_ids'], X_train_encoded['attention_mask'], y_train)
# test_data = TensorDataset(X_test_encoded['input_ids'], X_test_encoded['attention_mask'], y_test)

# train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
# test_loader = DataLoader(test_data, batch_size=16, shuffle=False)
# Tạo DataLoader cho tập train và test với chỉ số từ vựng
train_data = TensorDataset(X_train_indices, y_train)
test_data = TensorDataset(X_test_indices, y_test)

train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
test_loader = DataLoader(test_data, batch_size=16, shuffle=False)


In [12]:
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim=100, hidden_size=128, num_classes=6):
        super(LSTMClassifier, self).__init__()
        # Thêm lớp embedding
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_size, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, num_classes)  # Bidirectional nên hidden_size * 2

    def forward(self, input_indices):
        embedded = self.embedding(input_indices)  # Chuyển từ chỉ số từ vựng sang vector embedding
        lstm_output, (hn, cn) = self.lstm(embedded)
        # Chỉ lấy hidden state của lớp cuối cùng
        output = self.fc(lstm_output[:, -1, :])
        return output


vocab_size = len(vocab)  # Kích thước từ điển
model = LSTMClassifier(vocab_size=vocab_size, num_classes=6)

# Đưa mô hình sang GPU nếu có
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

LSTMClassifier(
  (embedding): Embedding(13004, 100)
  (lstm): LSTM(100, 128, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=256, out_features=6, bias=True)
)

In [13]:
# Khởi tạo các thành phần cho quá trình huấn luyện
optimizer = optim.Adam(model.parameters(), lr=1e-5)
criterion = nn.CrossEntropyLoss()

# Định nghĩa hàm huấn luyện với hiển thị tiến trình
def train_model(model, train_loader, criterion, optimizer, device, epochs=20):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        print(f"Epoch {epoch+1}/{epochs}")
        
        # Sử dụng tqdm để hiển thị tiến trình theo từng batch
        for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
            input_indices, labels = [x.to(device) for x in batch]

            # Reset gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(input_indices)  # Thay input_ids thành input_indices
            loss = criterion(outputs, labels)

            # Backpropagation
            loss.backward()
            optimizer.step()

            total_loss += loss.item()


        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1} completed with average loss: {avg_loss:.4f}")


In [14]:
train_model(model, train_loader, criterion, optimizer, device, epochs=20)

Epoch 1/20


Training Epoch 1: 100%|██████████| 3564/3564 [00:21<00:00, 167.34it/s]


Epoch 1 completed with average loss: 1.7922
Epoch 2/20


Training Epoch 2: 100%|██████████| 3564/3564 [00:20<00:00, 175.08it/s]


Epoch 2 completed with average loss: 1.7920
Epoch 3/20


...

Epoch 18 completed with average loss: 1.7919
Epoch 19/20


Training Epoch 19: 100%|██████████| 3564/3564 [00:21<00:00, 168.76it/s]


Epoch 19 completed with average loss: 1.7920
Epoch 20/20


Training Epoch 20: 100%|██████████| 3564/3564 [00:21<00:00, 168.79it/s]

Epoch 20 completed with average loss: 1.7920


In [15]:
def evaluate_model(model, test_loader, device):
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch in test_loader:
            input_indices, labels = [x.to(device) for x in batch]

            outputs = model(input_indices)  # Thay input_ids thành input_indices
            preds = torch.argmax(outputs, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    
    return all_labels, all_preds

y_true, y_pred = evaluate_model(model, test_loader, device)

print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      2354
           1       0.00      0.00      0.00      2428
           2       0.00      0.00      0.00      2364
           3       0.00      0.00      0.00      2386
           4       0.00      0.00      0.00      2358
           5       0.17      1.00      0.28      2365

    accuracy                           0.17     14255
   macro avg       0.03      0.17      0.05     14255
weighted avg       0.03      0.17      0.05     14255



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
# Hàm mã hóa câu văn bản đơn lẻ và thực hiện dự đoán
def classify_sentence(model, sentence, vocab, device, max_len=256):
    # Chuyển câu thành chỉ số từ vựng
    sentence_indices = torch.tensor([sentence_to_indices(sentence, vocab, max_len)]).to(device)
    
    # Đặt mô hình ở chế độ đánh giá (evaluation mode)
    model.eval()

    with torch.no_grad():
        # Dự đoán nhãn cho câu đầu vào
        outputs = model(sentence_indices)
        predicted_label = torch.argmax(outputs, dim=1).item()
    
    return predicted_label


In [19]:
sentence = "giao hàng đầy đủ đóng gói chắc đủ chắn"
predicted_label = classify_sentence(model, sentence, vocab, device)
print(f"Đánh giá: {predicted_label}")


Đánh giá: 5
